In [33]:
from jupyter_plotly_dash import JupyterDash

import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps


from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
username = "aacuser1"
password = "animals"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash("SimpleExample")

image_filename = 'Grazioso Salvare Logo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Luis Garcia Project #2 Dashboard'))),
    html.Hr(),
    # customer image location, can you put it in different location and resize?
    html.Img(id = 'customer-image', src = 'data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    
    html.Div(      
        dcc.RadioItems(
            id = 'filter-type',
            options = [
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value = 'reset'
        )    
    ),
    html.Hr(),
    
   
    #data Table
    dash_table.DataTable(
        id = 'datatable-interactivity',
        columns = [
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data = df.to_dict('records'),
        
        #interactive data table to make it user-friendly for your client
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = True,
        row_selectable = True,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),

    #set up the dashboard for the chart and geolocation side-by-side
    html.Div(className = 'row',
         style = {'display' : 'flex'},
             children =[       
        html.Div(
            id = 'graph-id',
            className = 'col s12 m6',
            ),       
        html.Div(
            id = 'map-id',
            className = 'col s12 m6',
            )
        ])
])
#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-interactivity','data'),
               Output('datatable-interactivity','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
        
        # filter for Water Rescue
        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
        # filter for Mountain/Wilderness Rescue
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
            
        # filter for Disaster Rescue/Individual Tracking
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            }))
            
        # Reset(no filter)
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

# Change the background for columns when selected
@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")])
def update_graphs(viewData):
    # code for chart
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(
            figure = px.histogram(dff, x = 'breed', title = 'Dog by Breed')
        )    
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data"),
     Input('datatable-interactivity', 'selected_rows'),
     Input('datatable-interactivity', 'selected_columns')])
def update_map(viewData, selected_rows, selected_columns):
    # imports the currently viewed data
    dff = pd.DataFrame.from_dict(viewData)
    #determines the default status
    if selected_rows == []:
        selected_rows = [0]
    # creats a map for a single selected row or the default
    if len(selected_rows) == 1:
        return [
            dl.Map(style={'width':'1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
            
                #marker with tool tip and popup
                dl.Marker(position=[(dff.iloc[selected_rows[0],13]), 
                                    (dff.iloc[selected_rows[0],14])], 
                          children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        html.H4("Animal Name"),
                        html.P(dff.iloc[selected_rows[0],9]),
                        
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows[0],12]),
                        
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows[0],4]),
                        
                        html.H4("Age"),
                        html.P(dff.iloc[selected_rows[0],15])
                    ])
                ])
            ])
        ]

app

login success
read success
